# Bend kinematics analysis

This code computes meander bend kinematics and compares with recorded kinematics on geological sections across bends.

### author:
Martin LEMAY
martin.lemay@mines-paris.org

### Related publication

If you use this script, please refer to the following publication:
- Lemay, M., Grimaud, J. L., Cojan, I., Rivoirard, J., & Ors, F. (in press) Submarine channel stacking patterns controlled by the autogenic 3D kinematics of meander bends. GSL special issue on meandering systems.

### Dependencies
- shapely 1.7.1
- dtw-python 1.1.10

## Python modules import

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns
sns.set()

from scipy.signal import savgol_filter
import scipy.stats as stats
import statsmodels.api as sm

import Centerline_collection
import plot_functions as plot
import centerline_process_function as cpf

import warnings
warnings.filterwarnings("ignore")

## Data import and section creation

### Import data and create centerline collections

In [ ]:
cdir = os.getcwd()
root = os.path.dirname(os.path.dirname(cdir))

root += "path/to/data/"
filepath_centerlines = "centreline_filename.csv"

# Parameters
avulsion_period = 4000
width           = 1200            # channel width
depth_max       = 50              # channel max depth
flow_dir        = np.array([1,0]) # flow direction vector

# resampling and bend detection parameters
spacing            = 100        # spacing between channel point
smooth_distance    = 1.5*width  # curvature and channel point location smoothing distance
sinuo_thres        = 1.05       # threshold for bends
lag                = 1          # minimum lag between 2 consecutive inflection points
apex_proba_ponds   = (1.,0.,0.) # curvature, amplitude, length
bend_evol_validity = 5          # minimum number of bends in a bend_evolution


# parse the file into one file per channel belt
centerlines_data = pd.read_csv(root + filepath_centerlines, sep=';')

all_keys = np.unique(centerlines_data["Iteration"])
prev_key = 0
for i, key in enumerate(np.arange(avulsion_period, all_keys.max()+1, avulsion_period)):
    mask = (centerlines_data["Iteration"] >= prev_key) & (centerlines_data["Iteration"] < key)
    extract = centerlines_data.loc[mask]
    extract.to_csv(root + filepath_centerlines + "_%s.csv"%(i), sep = ';', index=False)
    prev_key = key

# create the centerline collection objects from CenterlineProps files
cl_collec_all = []
domain = [[],[]]

print("Centerline importation in progress...")
for i in range(9):
    cl_collec_all += [Centerline_collection.Centerline_collection(
                      root + filepath_centerlines + "_%s.csv"%(i), spacing, smooth_distance,
                      lag=lag, sinuo_thres=sinuo_thres, width=width,
                      apex_proba_ponds=apex_proba_ponds,
                      compute_curvature=True, interpol_props=True, plot_curvature=False)]

print("Centerlines were imported.")

### Compute mean sinusity of last centerline for each channel belt

In [ ]:
mean_sinuo = 0
k = 0
for cl_collec in cl_collec_all:
    pt0 = cl_collec.centerlines[cl_collec.all_iter[-1]].cl_points[0].pt
    pt1 = cl_collec.centerlines[cl_collec.all_iter[-1]].cl_points[-1].pt
    delta_s = cl_collec.centerlines[cl_collec.all_iter[-1]].cl_points[-1].s

    mean_sinuo += (delta_s) / cpf.distance(pt0, pt1)
    k += 1

mean_sinuo /= k

print("Mean sinuosity of all centerlines: %.2f"%mean_sinuo)

### Track channel points along channel belt

In [ ]:
# channel points tracking (time warping) parameters
dmax = 0.5 * width # maximal distance between 2 consecutive points at iterations t and t+1
distance_weight = 0.33
vel_perturb_weight = 0.33
curvature_weight = 0.33
dtw_window = 5
dtw_pattern = "asymmetric"

print("Channel centerlines tracking in progress...")
for i, cl_collec in enumerate(cl_collec_all):
    print("  Centerlines from channel belt %s are being tracked..."%i)
    cl_collec.match_centerlines(dmax = dmax, distance_weight=distance_weight, 
                                vel_perturb_weight=vel_perturb_weight,
                                curvature_weight=curvature_weight,
                                window=dtw_window, pattern=dtw_pattern)
    
print("Channel centerlines tracking is done.")

### Create cross-sections

Section coordinates were manually defined such as the sections intersect once each centerline of a given channel belt.

In [ ]:
# Section coordinates

pts_start0 = [(-2000, 96000), (-500, 93000), (1000, 97500), (2500, 91000),
             (7000, 97500), (11000, 90500), (15000, 91000),
             (16200, 96000), (22000, 92500), (22000, 97000), (26000, 93000), (26500, 98000), (28500, 92000),
             (32500, 92000), (34000, 99500), (36250, 92000), (39000, 99500), (43000, 96000),
             (45750, 100500), (50500, 96500), (51500, 101500), (54000, 95000)]
pts_end0   = [(-2500, 93500), (-1000, 95500), (1000, 93500), (3500, 94000),
             (8000, 93500), (10500, 94500), (14500, 95000),
             (18000, 93000), (20000, 95000), (24000, 94000), (25000, 96500), (27500, 94500), (29500, 96500),
             (32500, 96500), (35000, 95500), (37500, 97000), (40000, 96500), (43000, 99000),
             (45500, 98000), (48500, 99000), (51500, 97500), (54000, 99000)]

pts_start1 = [(-5000, 45000), (0.000, 49500), (4000, 50250), (10000, 47250), (13000, 45750),
              (13500, 48500), (19500, 45500), (22800, 50000), (26800, 52000), (26250, 45000),
              (31250, 46500), (32000, 49500), (38250, 53500), (45000, 49500),
              (44250, 53500), (47400, 50400), (50500, 56000), (53500, 53000), (54250, 56000)]
pts_end1   = [(-1500, 48000), (1000, 48000), (3750, 48250), (10000, 48500), (12400, 48000),
              (14500, 47000), (21000, 48500), (23250, 48500), (24500, 47500), (28000, 49500),
              (30500, 48500), (32500, 48500), (37500, 49500), (43000, 51500),
              (46000, 50500), (47500, 53000), (50750, 53500), (52750, 55000), (54500, 53500)]

pts_start2 = [(-3000, 69500), (-3000, 63500), (1750, 70500), (5500, 68500), (7500, 72500),
              (8300, 67000), (12500, 72750), (12000, 66000), (15750, 71750), (18000, 68500),
              (22000, 73000), (23500, 67000), (25000, 73500), (30000, 71000), (28500, 68000),
              (32000, 71500), (34500, 70500), (37000, 73000), (38500, 69500), (39500, 75500),
              (45000, 76000), (45250, 68000), (48000, 76000), (50500, 71500),
              (53000, 74500), (54500, 70000)]
pts_end2   = [(-4500, 66500), (-2000, 67000), (2000, 68500), (4500, 71000), (6750, 69000),
              (9000, 70500), (11250, 69000), (13500, 69500), (15750, 69000), (18000, 70500),
              (22000, 69500), (24000, 71000), (26000, 71500), (27500, 70500), (31000, 70500),
              (33000, 70000), (34000, 71500), (36250, 70500), (38000, 72500), (40500, 71750),
              (44000, 72000), (46500, 73000), (49000, 72500), (51000, 73500),
              (52750, 71500), (54800, 73500)]

pts_start3 = [(-2500, 28500), (-3500, 21500), (1500, 27000), (6600, 26000),
              (6500, 21000), (17000, 27000), (16500, 21500), (20000, 27000),
              (23750, 21250), (29000, 19000), (33750, 22000), (39000, 16500),
              (41250, 19500), (43000, 17500), (45250, 21500), (47500, 17500),
              (49500, 23000), (51000, 18000), (52250, 23000)]
pts_end3   = [(-5000, 25000), (-1500, 25000), (500., 23500), (5000, 23500),
              (7750., 24000), (15000, 24000), (18000, 25500), (19750, 24000),
              (24000, 23500), (29500, 21000), (35500, 19000), (39000, 19500),
              (41250,17000), (42500, 20000), (45250, 19000), (47500, 21000),
              (49500, 19500), (51250, 21500), (53500, 20500)]

pts_start4 = [(2500, -1500), (5500, 1500), (5500, -4500), (10000, -2500), (15500, -4500),
              (13000, 500.), (18250, -4000), (24000, -3500), (23500, -8000), (26250, -500),
              (29500, -4500), (32750, -2000), (37000, -8000), (41250, -1500), (42750, -6500),
              (45250, -2500), (50000, -6500), (49000, -2000)]
pts_end4   = [(2500, 1000.), (4500, -1500), (7500, -2000), (10000, -4000), (12500, -2500),
              (16000, -3000), (18500, -2500), (23000, -5500), (25500, -4000), (28500, -4000),
              (30000, -2500), (31500, -4500), (37500, -4500), (41000, -5500), (43500, -2500),
              (45250, -5000), (47500, -3500), (50500, -3000)]

pts_start5 = [(0.000, 62000), (1500, 58000), (4000, 58500), (5500, 56000), (7750, 61000),
              (9750, 56500), (13000, 55000), (15000, 60000), (15500, 54500), (21000, 54500),
              (24000, 60000), (27000, 55000), (35500, 57500), (36000, 61000),
              (39000, 57000), (40000, 62000), (42500, 56500), (49000, 61000), (51500, 56500)]
pts_end5   = [(0.000, 58500), (2000, 59500), (3500, 57500), (5500, 59000), (8250, 58000),
              (10500, 58000), (12250, 58000), (14750, 56500), (17000, 57000), (19500, 57000),
              (25000, 56500), (27500, 57500), (34000, 59000), (36750, 58000),
              (38500, 60000), (40750, 58500), (42750, 59500), (48000, 58000), (50500, 59500)]

pts_start6 = [(2000., -1700), (4000., 4000.), (4000, -3000), (9000., 1500.), (12500, -1500),
              (8500., 6250.), (14000, 5000.), (18000, 10000), (19500, 4000.), (22500, 8500.),
              (25000, 6000.), (24500, 10000), (27000, 9500.), (28500, 12500), (30250, 9000),
              (32500, 12500), (32000, 7000.), (35750, 8500.), (35500, 5500.), (40500, 10500),
              (44000, 4000.), (45500, 10500), (49250, 5000.), (51750, 8500.), (52000, 11500)]
pts_end6   = [(1500., 1500.), (3500., 0.000), (6000, 500.0), (8750., -500.), (10000, 1500.),
              (11500, 4500.), (13500., 6500.), (18000, 6500.), (21000, 8500.), (22500, 6500.),
              (23500, 8500), (25250, 8500.), (26250, 11000), (28500, 10000), (30250, 11500),
              (31500, 9000), (33250, 9500.), (34500, 7000.), (36750, 8500.), (40500, 7500.),
              (43000, 8000.), (45750, 8000.), (49500, 8000.), (51750, 6000.), (54500, 8500.)]

pts_start7 = [(-1000, 78000), (4000., 83000), (7000., 81000), (6000., 85500), (10000, 83500),
              (7000., 88500), (14500, 85500), (15000, 90000), (16500, 85000), (21000, 84000),
              (21000, 90000), (23000, 82500), (30000, 87500), (29250, 83500),
              (32750, 87000), (34250, 84000), (37000, 88500), (37750, 83500), (41500, 88500),
              (43000, 85000), (43500, 89000), (46500, 88500), (46500, 92000), (52000, 85000),
              (54000, 90600)]
pts_end7   = [(-2500, 80000), (3750., 81500), (5000., 83500), (8000., 83000), (7500., 86000),
              (10500., 86000), (12500, 88000), (15500, 87000), (17250, 87000), (19000, 87500),
              (21750, 86000), (23500, 87000), (28000, 86000), (30750, 86500),
              (32500, 85000), (34500, 86500), (36500, 85000), (38500, 86500), (41000, 86000),
              (42750, 88000), (45000, 87500), (45000, 90000), (47500, 89000), (51500, 88500),
              (54000, 87000)]

pts_start8 = [(-1000, 98500.), (500.0, 105500), (4750., 101500), (6000., 104750),
              (11500, 104000), (16500, 103000), (16250, 99500.), (20000, 102000),
              (22500, 99000.), (24500, 104500), (27250, 100000), (42000, 106000),
              (48500, 100500), (50500, 106100)]
pts_end8   = [(0.000, 102500), (2500., 101500), (4000., 103500), (5750., 102000),
              (12000, 101500), (15000, 101000), (17750, 102000), (20000, 100500),
              (22500, 102000), (25000, 101000), (27000, 103500), (40750, 103000),
              (47500, 103000), (51000, 102500)]

# selection
# 2-9 : (18000, 68500), (18000, 70500)
# 6-5 : (8500., 6250.), (11500, 4500.)
# 6-14: (30250, 9000.), (30250, 11500)
# 9-6 : (16250, 99500.), (17750, 102000)

pts_start_all = (pts_start0, pts_start1, pts_start2, pts_start3, pts_start4,
                 pts_start5, pts_start6, pts_start7, pts_start8)
pts_end_all = (pts_end0, pts_end1, pts_end2, pts_end3, pts_end4,
               pts_end5, pts_end6, pts_end7, pts_end8)

print("Cross-section calculation in progress...")
for i, (cl_collec, pts_start, pts_end) in enumerate(zip(cl_collec_all, pts_start_all, pts_end_all)):
    cl_collec.set_section_lines(pts_start, pts_end)
    cl_collec.find_points_on_sections(thres=10, width=width, depth=depth_max, flow_dir=flow_dir, cl_collec_id=i)
    
print("Cross-sections were computed.")

### Display final centrelines (Figure 4)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8), dpi=150)
for i, cl_collec in enumerate(cl_collec_all):
    plot.plot_centerline_single("", (cl_collec_all[i].centerlines[cl_collec_all[i].all_iter[-1]].cl_points,),
                   cl_collec_all[i].centerlines[cl_collec_all[i].all_iter[-1]].bends,
                   domain=[[],[]], show = False, annotate = False,
                   plot_apex = False, plot_inflex = True, plot_middle = False,
                   plot_centroid = False, plot_apex_proba = False, plot_normal = False, scale_normal = 1.,
                   annot_text_size=10, color_bend=True, linewidth=1, markersize=2, ax=ax)

    pt = cl_collec.centerlines[cl_collec.all_iter[-1]].cl_points[0].pt
    ax.text(pt[0]-5000, pt[1], str(i), size=10)
    for j, section_line in enumerate(cl_collec.section_lines):
        coords = np.array(section_line)
        ax.plot((coords[0][0], coords[1][0]),
                (coords[0][1], coords[1][1]),
                'k-', linewidth=1)
        ax.text(coords[0][0], coords[0][1], str(j), size=8)

plt.axis('equal')
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.show()

## Stacking pattern type classification

In [ ]:
# Parameters
mig_threshold = 0.0125 * width # lateral distance threshold below which migration is considered null
frac_threshold = 0.95 # 
begin_threshold = 0.1 # 

print("Stacking pattern computation in progress...")

# Create DataFrame to store the results for all sections
# 0 = w ; 1 = f in related paper
len_cl_collec = [len(cl_collec.section_lines) for cl_collec in cl_collec_all]
nb_sections = sum(len_cl_collec)
columns = ("id", "bend_id",
           "migration_type", "Dx0", "Dz0", "Bcb0", "Hcb0", "Bcb_norm0", "Hcb_norm0", "Bcb_on_Hcb0", "Msb0",
           "Dx1", "Dz1", "Bcb1", "Hcb1", "Bcb_norm1", "Hcb_norm1", "Bcb_on_Hcb1", "Msb1",
           "DxApex", "DyApex", "DmigApex", "DzApex", "Mapex", "DxBend", "DyBend", "DmigBend", "DzBend", "Mbend")
data_sections = pd.DataFrame(np.full((nb_sections, len(columns)), np.nan),
                             columns=columns)

# Computation
st_types = ("1 way", "No mig + 1 way", "2 ways", "Multi ways")
index = 0
mig_classif = []
for i, cl_collec in enumerate(cl_collec_all):
    for j, section in enumerate(cl_collec.sections):

        mig_type = section.get_stacking_pattern_type(mig_threshold, frac_threshold, begin_threshold)
        mig_classif += [mig_type]
        data_sections.loc[index,"id"] = section.id
        data_sections.loc[index,"bend_id"] = section.bend_id
        data_sections["migration_type"][index] = mig_type
        index += 1

data_sections.set_index("id", drop=False, inplace=True)
print("Stacking pattern types were computed.")

In [ ]:
mig_classif = np.array(mig_classif)

nb_1way = np.sum(mig_classif == 0)
nb_1way_aggrad = np.sum(mig_classif == 1)
nb_2ways = np.sum(mig_classif == 2)
nb_multi_ways = np.sum(mig_classif == 3)

counts = [nb_1way, nb_1way_aggrad, nb_2ways, nb_multi_ways]
nb_tot = np.sum(counts)
counts = np.array(counts) / nb_tot

print("Total number of sections: %s"%(nb_tot))
fig, ax = plt.subplots(dpi=150)
plt.bar(st_types, counts)
plt.xlabel("Migration type")
plt.ylabel("Fraction")
plt.ylim(0,1)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(dpi=150)
plt.pie(counts, labels=st_types)
plt.tight_layout()
plt.show()

print("Stacking pattern types distribution:")
for i, st_type in enumerate(st_types):
    print("  %s: %.2f"%(st_type, counts[i] / counts.sum()))

## Recorded kinematics measurements

In [ ]:
print("Recorded channel displacements in progress...")

for i, cl_collec in enumerate(cl_collec_all):
    for j, section in enumerate(cl_collec.sections):
        section.channel_apparent_displacements(width, depth_max,
                                               False, "")
        section.section_averaged_channel_displacements(width, depth_max, mig_threshold,
                                                       False, "")

        data_sections.loc[section.id,("Dx0", "Dz0", "Bcb0", "Hcb0", "Bcb_norm0", "Hcb_norm0", "Bcb_on_Hcb0", "Msb0")] = section.averaged_disp["full"]
        data_sections.loc[section.id,("Dx1", "Dz1", "Bcb1", "Hcb1", "Bcb_norm1", "Hcb_norm1", "Bcb_on_Hcb1", "Msb1")] = section.averaged_disp["bend"]
        
        # Mobility numbers
        data_sections["Msb0"] = data_sections["Dx0"] / data_sections["Dz0"] * depth_max / width        
        data_sections["Msb1"] = data_sections["Dx1"] / data_sections["Dz1"] * depth_max / width

print("Recorded channel displacements where computed.")

## Real kinematics measurements

### Compute bend kinematics

In [ ]:
print("Bend kinematics computation in progress...")

dzmax = 78
bend_disp_all = {}
local_disp_all_bend = [[], [], [], []]
for i, cl_collec in enumerate(cl_collec_all):
    for j, section in enumerate(cl_collec.sections):            
        cl_idx, bend_idx = section.bend_id.split("-")
        bend = cl_collec.centerlines[cl_collec.all_iter[-1]].bends[eval(bend_idx)]

        pt_middle = 0.5 * (cl_collec.centerlines[cl_collec.all_iter[-1]].cl_points[bend.index_inflex_up].pt +
                           cl_collec.centerlines[cl_collec.all_iter[-1]].cl_points[bend.index_inflex_down].pt)
        pts_x = []
        pts_y = []
        pt_mig = [pt_middle]

        l_bend_pts = [pt_middle[:2]]
        l_index_new = np.arange(bend.index_inflex_up, bend.index_inflex_down+1, 1).astype(int)
        keys = cl_collec.all_iter[::-1]
        for k, key in enumerate(keys):
            if key == keys[-1]:
                break

            pts_x_iter = []
            pts_y_iter = []

            mig_mean = np.zeros(3)
            l_index = l_index_new
            l_index_new = []
            next_key = keys[k+1]
            for index in l_index:
                index = int(index)
                if np.isfinite(cl_collec.centerlines[key].index_cl_pts_prev_centerline[index]):
                    index_new = int(cl_collec.centerlines[key].index_cl_pts_prev_centerline[index])
                    pt_key = cl_collec.centerlines[key].cl_points[index].pt
                    pt_next_key = cl_collec.centerlines[next_key].cl_points[index_new].pt
                    mig_mean += (pt_key - pt_next_key)
                    l_index_new += [index_new]

                    if key == keys[0]:
                        pts_x_iter += [pt_key[0]]
                        pts_y_iter += [pt_key[1]]
                    pts_x_iter += [pt_next_key[0]]
                    pts_y_iter += [pt_next_key[1]]


            if len(l_index) > 0:
                mig_mean /= len(l_index)
                l_bend_pts += [l_bend_pts[-1] + mig_mean[:2]]

                if section.id in ("8-6", "6-5", "6-14", "2-9"):
                    bend_disp_all[section.id] = l_bend_pts

            pts_x += [pts_x_iter]
            pts_y += [pts_y_iter]
            pt_mig += [pt_mig[-1] + mig_mean]

        if len(l_bend_pts) > 30:

            l_bend_trajec_smooth = cpf.smooth_trajec(l_bend_pts[::-1], np.linspace(0, 3900, len(l_bend_pts)),
                              np.linspace(0, 3900, 40), 2, True)

            dir_trans = np.array(section.pt_start) - np.array(section.pt_stop)
            dir_trans /= np.linalg.norm(dir_trans)

            local_disp, whole_disp = cpf.compute_point_displacements(l_bend_trajec_smooth,
                                                cpf.perp(dir_trans[:2]), np.array((1., 0.)))

            local_disp_all_bend[0] += [section.id]
            local_disp_all_bend[1] += [savgol_filter(local_disp[:,3], 9, 2)]
            local_disp_all_bend[2] += [savgol_filter(section.local_disp[:,0], 9, 2)]
            curv = [isoline.cl_pt_ref.curvature() for isoline in section.isolines]
            local_disp_all_bend[3] += [curv]

            data_sections["DxBend"][section.id] = np.abs(whole_disp[0])
            data_sections["DyBend"][section.id] = np.abs(whole_disp[1])
            data_sections["DzBend"][section.id] = dzmax
            data_sections["DmigBend"][section.id] = np.abs(whole_disp[3])
            data_sections["Mbend"][section.id] = data_sections["DmigBend"][section.id] / data_sections["DzBend"][section.id] * depth_max/width

            if section.id in ("8-6", "6-5", "6-14", "2-9"):
                plt.figure(dpi=150)
                plot.plot_centerline_single("", (cl_collec.centerlines[keys[0]].cl_points,),
                      cl_collec.centerlines[keys[0]].bends[eval(bend_idx)-1:eval(bend_idx)+2],
                      domain=[[],[]], show = False, annotate = False,
                      plot_apex = False, plot_inflex = True, plot_middle = False,
                      plot_centroid = False, plot_apex_proba = False, plot_normal = False, scale_normal = 1.,
                      annot_text_size=10, color_bend=True, linewidth=1, markersize=2, ax=plt.gca())
                for k, pt in enumerate(l_bend_trajec_smooth):
                    if k == len(l_bend_trajec_smooth)-1:
                        break
                    plt.plot((pt[0], l_bend_trajec_smooth[k+1][0]), (pt[1], l_bend_trajec_smooth[k+1][1]), 'g-')
                plt.axis('equal')
                plt.title("Bend %s"%(section.id))
                plt.show()

print("Bend kinematics computation is done")

### Compute apex kinematics

In [ ]:
print("Apex kinematics computation in progress...")

local_disp_all = [[], [], [], []]
for i, cl_collec in enumerate(cl_collec_all):
    for j, section in enumerate(cl_collec.sections):

        cl_idx, bend_idx = section.bend_id.split("-")

        bend = cl_collec.centerlines[cl_collec.all_iter[-1]].bends[eval(bend_idx)]
        l_apex_pts = []
        index_apex = bend.index_apex
        for i, key in enumerate(cl_collec.all_iter[::-1]):
            if (not np.isfinite(index_apex)) or (key == cl_collec.all_iter[0]):
                break
            index_apex = int(index_apex)
            l_apex_pts += [cl_collec.centerlines[key].cl_points[index_apex].pt]

            if cl_collec.centerlines[key].index_cl_pts_prev_centerline[index_apex]:
                index_apex = cl_collec.centerlines[key].index_cl_pts_prev_centerline[index_apex]


        if len(l_apex_pts) > 30:
            l_apex_trajec_smooth = cpf.smooth_trajec(l_apex_pts[::-1], np.linspace(0, 3900, len(l_apex_pts)),
                                       np.linspace(0, 3900, 40), 2, True)

            dir_trans = np.array(section.pt_start) - np.array(section.pt_stop)
            dir_trans /= np.linalg.norm(dir_trans)

            local_disp, whole_disp = cpf.compute_point_displacements(l_apex_trajec_smooth,
                                                      cpf.perp(dir_trans[:2]), np.array((1., 0.)))

            local_disp_all[0] += [section.id]
            local_disp_all[1] += [savgol_filter(local_disp[:,3], 9, 2)]
            local_disp_all[2] += [savgol_filter(section.local_disp[:,0], 9, 2)]
            curv = [isoline.cl_pt_ref.curvature() for isoline in section.isolines]
            local_disp_all[3] += [curv]

            data_sections["DxApex"][section.id] = np.abs(whole_disp[0])
            data_sections["DyApex"][section.id] = np.abs(whole_disp[1])
            data_sections["DzApex"][section.id] = dzmax
            data_sections["DmigApex"][section.id] = np.abs(whole_disp[3])
            data_sections["Mapex"][section.id] = data_sections["DmigApex"][section.id] / data_sections["DzApex"][section.id] * depth_max/width
            
print("Apex kinematics computation is done")

### Compute all point kinematics

In [ ]:
print("All point kinematics computation in progress...")

local_disp_all_pts = [[], [], [], []]
for i, cl_collec in enumerate(cl_collec_all):
    for j, section in enumerate(cl_collec.sections):

        local_disp_all_pts[0] += [section.id]
        local_disp_all_pts[2] += [savgol_filter(section.local_disp[:,0], 9, 2)]
        curv = [isoline.cl_pt_ref.curvature() for isoline in section.isolines]
        local_disp_all_pts[3] += [curv]

        cl_idx, bend_idx = section.bend_id.split("-")
        bend = cl_collec.centerlines[cl_collec.all_iter[-1]].bends[eval(bend_idx)]
        l_index = np.arange(bend.index_inflex_up, bend.index_inflex_down+1, 1).astype(int)

        local_disp_all_pt = []
        for index in l_index:
            l_apex_pts = []
            for i, key in enumerate(cl_collec.all_iter[::-1]):
                if (not np.isfinite(index)) or (key == cl_collec.all_iter[0]):
                    break
                index = int(index)
                l_apex_pts += [cl_collec.centerlines[key].cl_points[index].pt]

                if cl_collec.centerlines[key].index_cl_pts_prev_centerline[index]:
                    index = cl_collec.centerlines[key].index_cl_pts_prev_centerline[index]


            if len(l_apex_pts) > 30:
                l_apex_trajec_smooth = cpf.smooth_trajec(l_apex_pts[::-1], np.linspace(0, 3900, len(l_apex_pts)),
                                           np.linspace(0, 3900, 40), 2, True)

                dir_trans = np.array(section.pt_start) - np.array(section.pt_stop)
                dir_trans /= np.linalg.norm(dir_trans)

                local_disp, whole_disp = cpf.compute_point_displacements(l_apex_trajec_smooth,
                                                          cpf.perp(dir_trans[:2]), np.array((1., 0.)))

                local_disp_all_pt += [savgol_filter(local_disp[:,3], 9, 2)]

        local_disp_all_pts[1] += [local_disp_all_pt]
        
print("All point kinematics computation is done.")

## Plot results

### Display selected bends (Figure 5)

In [ ]:
# selected bends and sections
lx2 = (12000, 10000, 24000, 8000, )
dom_y2 = (65000, 96000, 5000, -2000.)
cmap_name="viridis"

cl_collecs = (cl_collec_all[2], cl_collec_all[8], cl_collec_all[6], cl_collec_all[6])
sections_base = ("simu_2-section_9", "simu_8-section_6", "simu_6-section_14", "simu_6-section_5")

for i, cl_collec in enumerate(cl_collecs):

    # domain to plot
    domain = [[lx2[i], lx2[i]+10000],[dom_y2[i], dom_y2[i]+10000]]
    plot.plot_centerline_collection(False, cl_collec, domain, nb_cl=10, show = True,
                                    annotate = False, plot_apex = False, plot_inflex = False,
                                    plot_middle = False, plot_centroid = False, annot_text_size=10,
                                    color_bend = False, plot_apex_trajec = False,
                                    plot_centroid_trajec = False, plot_section = True, cmap_name=cmap_name,
                                    plot_warping = True)

### Plot sections (Figure 6)

In [ ]:
for i, cl_collec in enumerate(cl_collec_all):
    for j, section in enumerate(cl_collec.sections):
        fig, ax = plt.subplots(dpi=150)
        plot.plot_section(section, ax, np.array([1,0]), width, depth_max,
                          color_same_bend=True, colors=False, cmap=False)
        plt.title("Section %s"%(section.id))
        plt.tight_layout()
        plt.xlim(-4, 4)
        plt.ylim(-0.2, 5.2)
        plt.show()

### Recorded migration (Figures 8, 9b, 9d)

In [ ]:
# 0 = w ; 1 = f
for prop in ("Dx0", "Dx1", "Dz1", "Msb0", "Msb1"):

    if prop == "Dx0":
        ylim = (0, 4.0)
        xlabel = "Dxw"
        norm = width
    elif prop == "Dx1":
        ylim = (0, 4.)
        xlabel = "Dxf"
        norm = width
    elif prop == "Dz0":
        ylim = (0, 1.8)
        xlabel = "Dzw"
        norm = depth_max
    elif prop == "Dz1":
        ylim = (0, 1.8)
        xlabel = "Dzf"
        norm = depth_max
    elif prop == "Msb0":
        ylim = (0, 2.6)
        xlabel = "Msbw"
        norm = 1.
    elif prop == "Msb1":
        ylim = (0, 5)
        xlabel = "Msbf"
        norm = 1.

    ylabel = "Density"
    extract = data_sections[np.isfinite(data_sections[prop])]
    extract[prop] /= norm

    fig, ax = plt.subplots(dpi=150)
    sns.violinplot(x="migration_type", y=prop, data=extract, cut=0,
             inner="quartile")
    plt.ylabel(xlabel)
    plt.ylim(ylim[0], ylim[1])
    plt.tight_layout()
    plt.show()

### Bend average migration (Figures 7, 9e)

In [ ]:
for prop in ("DxBend", "DyBend", "DmigBend", "Mbend"):

    if prop == "DxBend":
        xlim = (0, 4.6)
        ylim = (0, 0.2)
        xlabel = "DxBend"
        norm = width
    elif prop == "DyBend":
        xlim = (0, 5)
        ylim = (0, 0.2)
        xlabel = "DyBend"
        norm = width
    elif prop == "DmigBend":
        xlim = (0, 4)
        ylim = (0, 0.2)
        xlabel = "DmigBend"
        norm = width
    elif prop == "Mbend":
        xlim = (0, 3)
        ylim = (0, 0.2)
        xlabel = "Mbend"
        norm = 1.
    
    ylabel = "Density"
    extract = data_sections[np.isfinite(data_sections[prop])]
    extract[prop] /= norm

    fig, ax = plt.subplots(dpi=150)
    sns.violinplot(x="migration_type", y=prop, data=extract, cut=0,
                  inner="quartile")

    plt.ylabel(xlabel)
    plt.tight_layout()
    plt.show()

    print("Bend kinematics:")
    for i in range(4):
        print("  Type %s:"%(i+1))
        print("    mean %s:   %.3f"%(prop, np.mean(extract[extract["migration_type"]==i][prop])))
        print("    median %s: %.3f"%(prop, np.median(extract[extract["migration_type"]==i][prop])))
        print()
    print()

    print("Bend kinematics:")
    print("  mean %s:   %.3f"%(prop, np.mean(extract[prop])))
    print("  median %s: %.3f"%(prop, np.median(extract[prop])))
    print()

### Bend vs stratigraphic mobility numbers

#### Correlation (Figures 9a, 9c)

In [ ]:
prop1 = "Mbend"
for prop2 in ("Msb0", "Msb1"):
    extract = data_sections[np.isfinite(data_sections[prop1])]

    fig, ax = plt.subplots(dpi=150)
    for mig_type, name in enumerate(("1 way", "No mig + 1 way", "2 ways", "Multi ways")):    
        # plot
        ax.plot(extract[extract["migration_type"]==mig_type][prop1],
                extract[extract["migration_type"]==mig_type][prop2],
                'o', label=name, markersize=2)

    err = (extract[extract["migration_type"]==mig_type][prop2] - extract[extract["migration_type"]==mig_type][prop1]).mean()
    print("Mean error %s: %s"%(name, err))

    err = ((extract[extract["migration_type"]==mig_type][prop2] - extract[extract["migration_type"]==mig_type][prop1])/extract[extract["migration_type"]==mig_type][prop1]).mean()
    print("Mean relative error %s: %s"%(name, err))

    plt.plot([0., 2.6], [0., 2.6], 'k--', linewidth=1)
    plt.xlabel(prop1)
    plt.ylabel(prop2)
    plt.xlim(0., 2.5)
    plt.ylim(0., 4.5)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # regression
    x = extract[prop2]
    y = extract[prop1]
    y = sm.add_constant(y)
    model = sm.OLS(x, y).fit()
    print("Summary view %s - Global"%prop2)
    print("p-value: %s"%(model.f_pvalue))
    print(model.summary())
    print()

#### Mobility ratios (Figure 9f)

In [ ]:
data_sections["Mobility_ratio0"] = data_sections["Mbend"] / data_sections["Msb0"]
data_sections["Mobility_ratio1"] = data_sections["Mbend"] / data_sections["Msb1"]


savedir = root + "figure_article/Mobility_ratio_"
for prop in ("Mobility_ratio0", "Mobility_ratio1"):
    fig, ax = plt.subplots(dpi=150)
    sns.violinplot(x="migration_type", y=prop, data=data_sections[np.isfinite(data_sections[prop])], cut=0,
                  inner="quartile")

    plt.ylabel(prop)
    plt.ylim(0, 4)
    plt.show()

# violin plot of mobility ratio 
extract = data_sections.loc[:, ["migration_type", "Mobility_ratio0", "Mobility_ratio1"]]
extract = extract[np.isfinite(extract["Mobility_ratio1"])]
extract["split"] = np.nan

toplot = pd.DataFrame(np.zeros((2*extract.index.size, 3)), columns=("migration_type", "Mobility_ratio", "split"))

toplot.loc[:extract.index.size-1, "migration_type"] = extract["migration_type"].tolist()
toplot.loc[extract.index.size:, "migration_type"] = extract["migration_type"].tolist()

toplot.loc[:extract.index.size-1, "Mobility_ratio"] = extract["Mobility_ratio0"].tolist()
toplot.loc[extract.index.size:, "Mobility_ratio"] = extract["Mobility_ratio1"].tolist()

toplot.loc[:extract.index.size-1, "split"] = "0"
toplot.loc[extract.index.size:, "split"] = "1"

fig, ax = plt.subplots(dpi=150)
sns.violinplot(x="migration_type", y="Mobility_ratio", data=toplot, cut=0,
                inner="quartile", split=True, hue="split")
plt.ylim(0, 4)
plt.show()

### Bend vs Apex mobility numbers (Supplementary material)

In [ ]:
prop1 = "Mbend"
prop2 = "Mapex"
extract = data_sections[np.isfinite(data_sections[prop1])]
extract = extract[np.isfinite(extract[prop2])]

savedir = root + "figure_article/Crossplot_" + prop2 + "_" + prop1

fig, ax = plt.subplots(dpi=150)
for mig_type, name in enumerate(("1 way", "Aggrad + 1 way", "2 ways", "Multi ways")):
    ax.plot(extract[extract["migration_type"]==mig_type][prop1],
          extract[extract["migration_type"]==mig_type][prop2],
          'o', label=name, markersize=2)

    err = (extract[extract["migration_type"]==mig_type][prop2] - extract[extract["migration_type"]==mig_type][prop1]).mean()
    print("Mean error %s: %s"%(name, err))

    err = ((extract[extract["migration_type"]==mig_type][prop2] - extract[extract["migration_type"]==mig_type][prop1])/extract[extract["migration_type"]==mig_type][prop1]).mean()
    print("Mean relative error %s: %s"%(name, err))

plt.plot([0., 2.6], [0., 2.6], 'k--', linewidth=1)
plt.xlabel(prop1)
plt.ylabel(prop2)
plt.xlim(0., 2.6)
plt.ylim(0., 2.6)
plt.legend()
plt.tight_layout()
plt.show()


x = extract[prop2]
y = extract[prop1]
y = sm.add_constant(y)
model = sm.OLS(x, y).fit()
print("Summary view %s - Global"%prop2)
print("p-value: %s"%(model.f_pvalue))
print(model.summary())
print()

### Local displacements (Figure 10)

In [ ]:
# bend mean kinematics + all points

for ide, pts_disp, record_disp, curv in zip(local_disp_all_pts[0], local_disp_all_pts[1], local_disp_all_pts[2], local_disp_all_pts[3]):

    if (ide not in ("8-6", "6-5", "6-14", "2-9")):
        continue

    if (len(pts_disp) == 0):
        continue

    apex_disp = np.array([])
    for ide1, apex_disp, record_disp1, curv1 in zip(local_disp_all_bend[0], local_disp_all_bend[1], local_disp_all_bend[2], local_disp_all_bend[3]):
        if ide1 == ide:
            break

    plt.figure(dpi=150)
    for pt_disp in pts_disp:
        plt.plot(pt_disp / width, np.arange(len(pt_disp)), '-', color='lightgrey', linewidth=0.3)
    plt.plot(apex_disp / width, np.arange(len(apex_disp)), 'k-')

    mig_pos = []
    age_pos = []
    mig_neg = []
    age_neg = []
    prev_curv = 1
    maxi = min(len(apex_disp), len(record_disp))

    for i in range(maxi):
        if curv[i] >= 0:
            if (prev_curv > 0) & (len(mig_pos)>0):
                mig_pos[-1] += [record_disp[i]/width]
                age_pos[-1] += [i]
            else:
                mig_pos += [[record_disp[i]/width]]
                age_pos += [[i]]
                if (len(mig_neg)>0):
                    mig_neg[-1] += [record_disp[i]/width]
                    age_neg[-1] += [i]
            prev_curv = 1

        if curv[i] <= 0:
            if prev_curv < 0 & (len(mig_neg)>0):
                mig_neg[-1] += [record_disp[i]/width]
                age_neg[-1] += [i]
            else:
                mig_neg += [[record_disp[i]/width]]
                age_neg += [[i]]
                if (len(mig_pos)>0):
                    mig_pos[-1] += [record_disp[i]/width]
                    age_pos[-1] += [i]
            prev_curv = -1

    xmin = 0
    for toplot_x, toplot_y in zip(mig_pos, age_pos):
        toplot_x = np.array(toplot_x)
        toplot_y = np.array(toplot_y)
        mask_pos = toplot_x >= 0
        mask_neg = toplot_x <= 0
        plt.plot(toplot_x[mask_pos], toplot_y[mask_pos], 'b-')
        plt.plot(np.abs(toplot_x[mask_neg]), toplot_y[mask_neg], 'b--')
        
    for toplot_x, toplot_y in zip(mig_neg, age_neg):
        toplot_x = np.array(toplot_x)
        toplot_y = np.array(toplot_y)
        mask_pos = toplot_x >= 0
        mask_neg = toplot_x <= 0
        plt.plot(toplot_x[mask_pos], toplot_y[mask_pos], 'r-')
        plt.plot(np.abs(toplot_x[mask_neg]), toplot_y[mask_neg], 'r--')

    plt.title("Section %s"%ide)
    plt.xlim(0, 0.22)
    plt.ylim(0, maxi)
    plt.xlabel("Migration")
    plt.ylabel("Iteration (x100)")
    plt.tight_layout()
    plt.show()